In [2]:
import pickle

import os
import random

def subfiles(folder,res, join=True, prefix=None, suffix=None, sort=True):
    # if join:
    #     l = os.path.join
    # else:
    #     l = lambda x, y: y
    # for i in os.listdir(folder):
    #     print(i)
    # res = [l(folder, i) for i in os.listdir(folder) if os.path.isfile(os.path.join(folder, i))
    #         and (prefix is None or i.startswith(prefix))
    #         and (suffix is None or i.endswith(suffix))]
    
    dirList=[]
    for i in os.listdir(folder):
        wholepath = os.path.join(folder, i)
        if os.path.isdir(wholepath):
            dirList.append(wholepath)
        if os.path.isfile(wholepath):
            res.append(wholepath)
            if not wholepath.endswith(suffix):
                res.remove(wholepath)
    if dirList:
        for subDir in dirList:
            subfiles(subDir,res,join=False,suffix="_preprocessed.npz")
    if sort:
        res.sort()



def create_splits(image_dir='/home/jovyan/main/BraTS2020_TrainingData/'):
    """File to split the dataset into multiple folds and the train, validation and test set.
    :param image_dir: Directory where the images lie in.
    """
    npz_files = []
    subfiles(image_dir,npz_files, suffix="_preprocessed.npz", join=False)# npy->npz
    sample_size = len(npz_files)

    testset_size = int(sample_size * 0.2)#0.25->0.2
    valset_size = int(sample_size * 0.2)
    trainset_size = sample_size - valset_size - testset_size  # Assure all samples are used.

    if sample_size < (trainset_size + valset_size + testset_size):
        raise ValueError("Assure more total samples exist than train test and val samples combined!")

    splits = []
    sample_set = {sample[:-4] for sample in npz_files.copy()}  # Remove the file extension
    test_samples = random.sample(sample_set, testset_size)  # IMO the Testset should be static for all splits

    for split in range(0, 5):
        train_samples = random.sample(sample_set - set(test_samples), trainset_size)
        val_samples = list(sample_set - set(train_samples) - set(test_samples))

        train_samples.sort()
        val_samples.sort()

        split_dict = dict()
        split_dict['train'] = train_samples
        split_dict['val'] = val_samples
        split_dict['test'] = test_samples

        splits.append(split_dict)

    # Todo: IMO it is better to write that dict as JSON. This (unlike pickle) allows the user to inspect the file with an editor
    with open(os.path.join(image_dir, 'splits.pkl'), 'wb') as f:
        pickle.dump(splits, f)

    splits_sanity_check(image_dir)


# ToDo: The naming "splits.pkl should not be distributed over multiple files. This makes changing of it less clear.
#   Instead move saving and loading to one file. (Here would be a good place)
#   Other usages are: spleen/create_splits.py:57 (Which is redundand anyways?);
#   UNetExperiment3D.py:55  and UNetExperiment.py:55
def splits_sanity_check(path):
    """ Takes path to a splits file and verifies that no samples from the test dataset leaked into train or validation.
    :param path
    """
    with open(os.path.join(path, 'splits.pkl'), 'rb') as f:
        splits = pickle.load(f)
        for i in range(len(splits)):
            samples = splits[i]
            tr_samples = set(samples["train"])
            vl_samples = set(samples["val"])
            ts_samples = set(samples["test"])

            assert len(tr_samples.intersection(vl_samples)) == 0, "Train and validation samples overlap!"
            assert len(vl_samples.intersection(ts_samples)) == 0, "Validation and Test samples overlap!"
            assert len(tr_samples.intersection(ts_samples)) == 0, "Train and Test samples overlap!"
    return

create_splits()

/tmp/ipykernel_91/647576444.py:51: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_samples = random.sample(sample_set, testset_size)  # IMO the Testset should be static for all splits
/tmp/ipykernel_91/647576444.py:54: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  train_samples = random.sample(sample_set - set(test_samples), trainset_size)
